In [ ]:
import yfinance as yf


ticker = yf.Ticker("TSLA")
symbol = ticker.info["symbol"]
name = ticker.info["displayName"]
df = ticker.history(period="1y")
df.head()

In [ ]:
"""Analyze stock trends to partition it into multiple parts"""

import numpy as np

from src.stock_gone_wrong.signal import (find_largest_changes,
                                         find_stock_peaks, remove_saddle)

data = df["Close"].to_numpy()
maxima = find_stock_peaks(data, 2)
minima = find_stock_peaks(-data, 2)

extrema = np.unique(np.concat(([0, data.size - 1], minima, maxima)))
trends = remove_saddle(data, extrema)
changes = find_largest_changes(data, trends, 5)

In [ ]:
"""Visualize the partition results, and highlight major upturns/downturns"""

from matplotlib import pyplot as plt


plt.figure(figsize=(10, 3))
mid_y = (data.min() + data.max()) / 2
bbox = {
    "facecolor": "#ECF5F5",
    "edgecolor": "black",
    "boxstyle": "round,pad=0.3",
}
for i, (x, y) in enumerate(changes):
    plt.axvspan(
        df.index[x],
        df.index[y],
        facecolor="lightgray",
        edgecolor="#434747",
        alpha=0.5,
        zorder=-10,
    )
    mid_x = df.index[x] + (df.index[y] - df.index[x]) / 2
    plt.text(mid_x, mid_y, f"{i+1}", ha="center", va="center", zorder=20, bbox=bbox)
plt.plot(df["Close"], label="Close")
plt.plot(df["Close"].iloc[trends], linestyle=":", label="Trend")
plt.scatter(
    df.index[minima], data[minima], s=15, color="#BC2020", zorder=10, label="Trough"
)
plt.scatter(
    df.index[maxima], data[maxima], s=15, color="#42BC20", zorder=10, label="Peak"
)
plt.title(f"{symbol} Stock Trend")
plt.legend()
plt.show()

In [ ]:
from stock_gone_wrong.search_news import search_news


change = changes[0]
search_results = search_news(
    f"{name}({symbol}) stock price", df.index[change[0]], df.index[change[1]]
)
for r in search_results[:5]:
    print(r)

In [ ]:
from langchain_ollama import ChatOllama, OllamaEmbeddings

try:
    chat = ChatOllama(model="granite3.2:8b")
    embeddings = OllamaEmbeddings(model="granite3.2:8b")
    response = chat.stream("Who was the first man on the moon?")
    for r in response:
        print(r.content, end="")
except:
    print(
        "Error in using ollama model. Please make sure it is opened and the model is available"
    )

In [ ]:
from stock_gone_wrong.agent import load_links

links: list[str] = [r.url for r in search_results]
vectorstore = load_links(links, embeddings)

In [ ]:
from stock_gone_wrong.agent import model_qa

verb = "rise" if data[change[0]] < data[change[1]] else "drop"
query = f"Why does the stock price of {name}({symbol}) {verb}?"
response, similar_docs = model_qa(chat, query, vectorstore)

print(
    f"Event: {ticker} {verb}s from {data[change[0]]} in {df.index[change[0]]} to {data[change[1]]} in {df.index[change[1]]}"
)
print("Query:", query)
print("Response:", response)
print("\nSources:")
for doc in similar_docs:
    print("-", doc.metadata)